<h1>GLI3: Demurger et al 2015</h1>
<p>Extract the clinical data from <a href="https://pubmed.ncbi.nlm.nih.gov/24736735/"target="__blank">Démurger F, et al. (2015) New insights into genotype-phenotype correlation for GLI3 mutations. Eur J Hum Genet ;23(1):92-102. PMID:24736735</a>.<p>
<p>Data were extracted from Tables 1 and 2.</p>
<p>Table 1 (and Supplemental Table 1) present data for Greig cephalopolysyndactyly syndrome (GCPS; MIM# 175700).</p>
<p>Table 2 (and Supplemental Table 2) present data for Pallister–Hall syndrome (PHS; MIM# 146510).</p>

# Greig cephalopolysyndactyly syndrome (GCPS; MIM:175700)

c.1543_1544dup was found in two affected sibs, was present at low level in DNA extracted from blood of their father (Family G068), suggesting a somatic mosaicism. We therefore remove the row corresponding to the father from further analysis.

Along the same line, a FISH analysis revealed a GLI3 deletion in only 56% of blood cells of a patient (G059) with bilateral preaxial PD of the feet and developmental delay. At least two patients (G005 and G019) had Greig cephalopolysyndactyly contiguous gene syndrome (GCPS-CGS) caused by haploinsufficiency of GLI3 and adjacent genes confirmed by array-CGH with a deletion of 7 and 9 Mb, respectively.

These individuals were also removed from the analysis because of the multifactorial pathophysiology.

We removed the corresponding rows from the following table.

In [1]:
from pyphetools.creation import TemplateImporter
from pyphetools.visualization import QcVisualizer, IndividualTable
from IPython.display import HTML, display
import pyphetools
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.9.63


In [2]:
hp_json = "../hp.json"
created_by="ORCID:0000-0002-0736-9199"
template = "input/GLI3_GCPS_individuals.xlsx"
timporter = TemplateImporter(hp_json=hp_json, template=template, created_by=created_by)

In [3]:
deletions = {"rsa7p14.1(kit P179)x1",
            "46,XY.ish del(7)(p14.1)(RP11-816F16-)",
            "46,XX.ish del(7)(p14.1p14.1)(GLI3-)"}
individuals, cvalidator = timporter.import_phenopackets_from_template(deletions=deletions)

HPO version 2024-02-27
Created encoders for 40 fields
 we got OMIM:175700  NM_000168.6
We output 51 GA4GH phenopackets to the directory phenopackets


In [4]:
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [5]:
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
G029 (UNKNOWN; n/a),Greig cephalopolysyndactyly syndrome (OMIM:175700),NM_000168.6:c.327del (heterozygous),Syndactyly (HP:0001159); Preaxial foot polydactyly (HP:0001841); excluded: Macrocephaly (HP:0000256); excluded: Agenesis of corpus callosum (HP:0001274); excluded: Hypoplasia of the corpus callosum (HP:0002079); excluded: Ventriculomegaly (HP:0002119); excluded: Hypertelorism (HP:0000316); excluded: Y-shaped metacarpals (HP:0006042); excluded: Y-shaped metatarsals (HP:0010567); excluded: Anal atresia (HP:0002023); excluded: Nail dysplasia (HP:0002164); excluded: Postaxial hand polydactyly (HP:0001162); excluded: Postaxial foot polydactyly (HP:0001830); excluded: Preaxial hand polydactyly (HP:0001177)
G070 (UNKNOWN; n/a),Greig cephalopolysyndactyly syndrome (OMIM:175700),NM_000168.6:c.427G>T (heterozygous),Macrocephaly (HP:0000256); Postaxial hand polydactyly (HP:0001162); Preaxial foot polydactyly (HP:0001841); excluded: Syndactyly (HP:0001159); excluded: Y-shaped metacarpals (HP:0006042); excluded: Y-shaped metatarsals (HP:0010567); excluded: Anal atresia (HP:0002023); excluded: Nail dysplasia (HP:0002164); excluded: Postaxial foot polydactyly (HP:0001830); excluded: Preaxial hand polydactyly (HP:0001177)
G070_Mother (UNKNOWN; n/a),Greig cephalopolysyndactyly syndrome (OMIM:175700),NM_000168.6:c.427G>T (heterozygous),excluded: Macrocephaly (HP:0000256); excluded: Hypertelorism (HP:0000316); excluded: Syndactyly (HP:0001159); excluded: Y-shaped metacarpals (HP:0006042); excluded: Y-shaped metatarsals (HP:0010567); excluded: Anal atresia (HP:0002023); excluded: Nail dysplasia (HP:0002164); excluded: Postaxial hand polydactyly (HP:0001162); excluded: Postaxial foot polydactyly (HP:0001830); excluded: Preaxial hand polydactyly (HP:0001177); excluded: Preaxial foot polydactyly (HP:0001841)
G118 (UNKNOWN; n/a),Greig cephalopolysyndactyly syndrome (OMIM:175700),NM_000168.6:c.444C>A (heterozygous),Macrocephaly (HP:0000256); Broad thumb (HP:0011304); Syndactyly (HP:0001159); Preaxial foot polydactyly (HP:0001841); excluded: Broad hallux (HP:0010055); excluded: Y-shaped metacarpals (HP:0006042); excluded: Y-shaped metatarsals (HP:0010567); excluded: Anal atresia (HP:0002023); excluded: Nail dysplasia (HP:0002164); excluded: Postaxial hand polydactyly (HP:0001162); excluded: Postaxial foot polydactyly (HP:0001830); excluded: Preaxial hand polydactyly (HP:0001177)
G13684 (UNKNOWN; n/a),Greig cephalopolysyndactyly syndrome (OMIM:175700),NM_000168.6:c.444C>A (heterozygous),Macrocephaly (HP:0000256); Hypertelorism (HP:0000316); Syndactyly (HP:0001159); Preaxial foot polydactyly (HP:0001841); excluded: Y-shaped metacarpals (HP:0006042); excluded: Y-shaped metatarsals (HP:0010567); excluded: Anal atresia (HP:0002023); excluded: Nail dysplasia (HP:0002164); excluded: Postaxial hand polydactyly (HP:0001162); excluded: Postaxial foot polydactyly (HP:0001830); excluded: Preaxial hand polydactyly (HP:0001177)
G13684_Brother (UNKNOWN; n/a),Greig cephalopolysyndactyly syndrome (OMIM:175700),NM_000168.6:c.444C>A (heterozygous),Hypertelorism (HP:0000316); Syndactyly (HP:0001159); Preaxial foot polydactyly (HP:0001841); excluded: Macrocephaly (HP:0000256); excluded: Y-shaped metacarpals (HP:0006042); excluded: Y-shaped metatarsals (HP:0010567); excluded: Anal atresia (HP:0002023); excluded: Nail dysplasia (HP:0002164); excluded: Postaxial hand polydactyly (HP:0001162); excluded: Postaxial foot polydactyly (HP:0001830); excluded: Preaxial hand polydactyly (HP:0001177)
G13684_Mother (UNKNOWN; n/a),Greig cephalopolysyndactyly syndrome (OMIM:175700),NM_000168.6:c.444C>A (heterozygous),Macrocephaly (HP:0000256); Syndactyly (HP:0001159); Preaxial foot polydactyly (HP:0001841); excluded: Y-shaped metacarpals (HP:0006042); excluded: Y-shaped metatarsals (HP:0010567); excluded: Anal atresia (HP:0002023); excluded: Nail dysplasia (HP:0002164); excluded: Postaxial hand polydactyly (HP:0001162); excluded: Postax

In [6]:
PMID = "PMID:24736735"
df = timporter.create_hpoa_from_phenopackets(pmid=PMID, moi="Autosomal dominant", target="OMIM:175700")

[pyphetools] Ingested 81 GA4GH phenopackets.
[INFO] Extracted 51 from 81 phenopackets with OMIM:175700

	Greig cephalopolysyndactyly syndrome (OMIM:175700): n=51
We found a total of 24 unique HPO terms
Extracted disease: Greig cephalopolysyndactyly syndrome (OMIM:175700)
Wrote HPOA disease file to OMIM-175700.tab


In [7]:
pmid = "PMID:24736735"
timporter.create_hpoa_from_phenopackets(pmid=pmid, moi="Autosomal dominant", target="OMIM:175700")

[pyphetools] Ingested 81 GA4GH phenopackets.
[INFO] Extracted 51 from 81 phenopackets with OMIM:175700

	Greig cephalopolysyndactyly syndrome (OMIM:175700): n=51
We found a total of 24 unique HPO terms
Extracted disease: Greig cephalopolysyndactyly syndrome (OMIM:175700)
Wrote HPOA disease file to OMIM-175700.tab


,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0000256,Macrocephaly,,,28/48,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199
1,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0001162,Postaxial hand polydactyly,,,26/51,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199
2,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0001841,Preaxial foot polydactyly,,,37/51,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199
3,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0000316,Hypertelorism,,,18/42,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199
4,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0001159,Syndactyly,,,32/49,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199
5,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0006042,Y-shaped metacarpals,,,0/51,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199
6,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0010567,Y-shaped metatarsals,,,0/51,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199
7,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0002023,Anal atresia,,,0/51,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199
8,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0002164,Nail dysplasia,,,0/51,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199
9,OMIM:175700,Greig cephalopolysyndactyly syndrome,HP:0001830,Postaxial foot polydactyly,,,11/51,,,,,PMID:24736735,PCS,ORCID:0000-0002-0736-9199


<h1> Pallister–Hall syndrome (PHS; MIM# 146510)</h1>
<p>The second half of this notebook extracts data about PHS from supplemental table 2.</p>

In [8]:
hp_json = "../hp.json"
created_by="ORCID:0000-0002-0736-9199"
template = "input/GLI3_PHS_individuals.xlsx"
timporter_PHS = TemplateImporter(hp_json=hp_json, template=template, created_by=created_by)

In [9]:
individuals_PHS, cvalidator_PHS = timporter_PHS.import_phenopackets_from_template()

HPO version 2024-02-27
Created encoders for 28 fields
 we got OMIM: 146510  NM_000168.6
We output 21 GA4GH phenopackets to the directory phenopackets


In [10]:
qc = QcVisualizer(cohort_validator=cvalidator_PHS)
display(HTML(qc.to_summary_html()))

In [11]:
table = IndividualTable(individuals_PHS)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
P15112 (UNKNOWN; n/a),Pallister-Hall syndrome (OMIM: 146510),NM_000168.6:c.1995del (heterozygous),Shortening of all distal phalanges of the fingers (HP:0006118); Y-shaped metacarpals (HP:0006042); Y-shaped metatarsals (HP:0010567); Hypothalamic hamartoma (HP:0002444); Anal atresia (HP:0002023); Bifid epiglottis (HP:0010564); Nail dysplasia (HP:0002164); excluded: Growth delay (HP:0001510); excluded: Postaxial polydactyly (HP:0100259); excluded: Preaxial hand polydactyly (HP:0001177); excluded: Preaxial foot polydactyly (HP:0001841); excluded: Intellectual disability (HP:0001249)
G097 (UNKNOWN; n/a),Pallister-Hall syndrome (OMIM: 146510),NM_000168.6:c.2072del (heterozygous),Growth delay (HP:0001510); Postaxial polydactyly (HP:0100259); Y-shaped metacarpals (HP:0006042); Y-shaped metatarsals (HP:0010567); Hypothalamic hamartoma (HP:0002444); excluded: Preaxial hand polydactyly (HP:0001177); excluded: Preaxial foot polydactyly (HP:0001841); excluded: Anal atresia (HP:0002023); excluded: Intellectual disability (HP:0001249)
G085 (UNKNOWN; n/a),Pallister-Hall syndrome (OMIM: 146510),NM_000168.6:c.2123_2126del (heterozygous),Postaxial polydactyly (HP:0100259); Y-shaped metacarpals (HP:0006042); Y-shaped metatarsals (HP:0010567); Nail dysplasia (HP:0002164); excluded: Growth delay (HP:0001510); excluded: Preaxial hand polydactyly (HP:0001177); excluded: Preaxial foot polydactyly (HP:0001841); excluded: Anal atresia (HP:0002023); excluded: Intellectual disability (HP:0001249)
G085-Father (UNKNOWN; n/a),Pallister-Hall syndrome (OMIM: 146510),NM_000168.6:c.2123_2126del (heterozygous),Growth delay (HP:0001510); Postaxial polydactyly (HP:0100259); Y-shaped metacarpals (HP:0006042); Y-shaped metatarsals (HP:0010567); excluded: Preaxial hand polydactyly (HP:0001177); excluded: Preaxial foot polydactyly (HP:0001841); excluded: Anal atresia (HP:0002023); excluded: Intellectual disability (HP:0001249)
G085-Aunt (UNKNOWN; n/a),Pallister-Hall syndrome (OMIM: 146510),NM_000168.6:c.2123_2126del (heterozygous),Postaxial polydactyly (HP:0100259); Shortening of all distal phalanges of the fingers (HP:0006118); Nail dysplasia (HP:0002164); excluded: Preaxial hand polydactyly (HP:0001177); excluded: Preaxial foot polydactyly (HP:0001841); excluded: Anal atresia (HP:0002023); excluded: Intellectual disability (HP:0001249)
G085-Grand-mother (UNKNOWN; n/a),Pallister-Hall syndrome (OMIM: 146510),NM_000168.6:c.2123_2126del (heterozygous),Postaxial polydactyly (HP:0100259); Nail dysplasia (HP:0002164); excluded: Preaxial hand polydactyly (HP:0001177); excluded: Preaxial foot polydactyly (HP:0001841); excluded: Anal atresia (HP:0002023); excluded: Intellectual disability (HP:0001249)
G121 (UNKNOWN; n/a),Pallister-Hall syndrome (OMIM: 146510),NM_000168.6:c.2149_2150insT (heterozygous),Postaxial polydactyly (HP:0100259); Y-shaped metacarpals (HP:0006042); Y-shaped metatarsals (HP:0010567); excluded: Growth delay (HP:0001510); excluded: Preaxial hand polydactyly (HP:0001177); excluded: Preaxial foot polydactyly (HP:0001841); excluded: Anal atresia (HP:0002023); excluded: Intellectual disability (HP:0001249); excluded: Nail dysplasia (HP:0002164)
G001 (UNKNOWN; n/a),Pallister-Hall syndrome (OMIM: 146510),NM_000168.6:c.2149C>T (heterozygous),Growth delay (HP:0001510); Postaxial polydactyly (HP:0100259); Shortening of all distal phalanges of the fingers (HP:0006118); Y-shaped metacarpals (HP:0006042); Y-shaped metatarsals (HP:0010567); Hypothalamic hamartoma (HP:0002444); Anal atresia (HP:0002023); excluded: Preaxial hand polydactyly (HP:0001177); excluded: Preaxial foot polydactyly (HP:0001841); excluded: Intellectual disability (HP:0001249)
G083 (UNKNOWN; n/a),Pallister-Hall syndrome (OMIM: 146510),NM_000168.6:c.2385del (heterozygous),Postaxial polydactyly (HP:0100259); Y-shaped metacarpals (HP:0006042); Y-shaped metatarsals (HP:0010567); excluded: Growth delay (HP:0001510); excluded: Preaxial hand polydactyly (HP:0

In [12]:
pmid = "PMID:24736735"
timporter.create_hpoa_from_phenopackets(pmid=pmid, moi="Autosomal dominant", target="OMIM:146510")

[pyphetools] Ingested 81 GA4GH phenopackets.
[INFO] Extracted 0 from 81 phenopackets with OMIM:146510

We found a total of 0 unique HPO terms


ValueError: Could not retrieve Disease for cohort